In [1]:
import pandas as pd
import numpy as np
import datetime

### EDA (análisis exploratorio de datos) donde se normaliza el tipo de datos de cada columna, se realizan las imputaciones correspondientes y se realizan las primeras visualizaciones para comprender los datos, aplicando filtros correspondientes a partir del archivo 'siniestrosETL.csv'

### Además se analiza la presencia de valores nulos, duplicados y outliers

In [2]:
siniestros = pd.read_csv('siniestrosETL.csv')

In [3]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28306 entries, 0 to 28305
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          28306 non-null  object
 1   FECHA       28306 non-null  object
 2   A           28306 non-null  int64 
 3   M           28306 non-null  int64 
 4   D           28306 non-null  int64 
 5   H           28306 non-null  int64 
 6   COMUNA      28130 non-null  object
 7   LONGITUD    28038 non-null  object
 8   LATITUD     28038 non-null  object
 9   TIPO_CALLE  28306 non-null  object
 10  ACUSADO     28306 non-null  object
 11  VICTIMA     28306 non-null  object
 12  FATAL       28306 non-null  int64 
dtypes: int64(5), object(8)
memory usage: 2.8+ MB


### Tratamiento de nulos

Se eliminan registros cuyos valores de COMUNA son nulos o sin dato

In [4]:
siniestros.COMUNA.unique()

array(['8', '9', '1', '11', '15', '4', '7', '12', '3', '13', '14', '10',
       '6', '2', '5', '0', 'No Especificada', nan, 'SD'], dtype=object)

In [5]:
siniestros = siniestros[(siniestros['COMUNA'] != 'No Especificada') &
                        (siniestros['COMUNA'] != 'SD') &
                        (siniestros['COMUNA'] != '0')]

In [6]:
siniestros = siniestros.dropna(subset=['COMUNA'])

Se pasa COMUNA a tipo de dato entero (int)

In [7]:
siniestros = siniestros.copy()
siniestros['COMUNA'] = siniestros['COMUNA'].astype(int)

In [8]:
siniestros

,ID,FECHA,A,M,D,H,COMUNA,LONGITUD,LATITUD,TIPO_CALLE,ACUSADO,VICTIMA,FATAL
0,2016-0001,2016-01-01,2016,1,1,4,8,-58.47533969,-34.68757022,AVENIDA,AUTO,MOTO,1
1,2016-0002,2016-01-02,2016,1,2,1,9,-58.50877521,-34.66977709,GRAL PAZ,PASAJEROS,AUTO,1
2,2016-0003,2016-01-03,2016,1,3,7,1,-58.39040293,-34.63189362,AVENIDA,AUTO,MOTO,1
3,2016-0004,2016-01-10,2016,1,10,0,8,-58.46503904,-34.68092974,AVENIDA,SD,MOTO,1
4,2016-0005,2016-01-21,2016,1,21,5,1,-58.38718297,-34.62246630,AVENIDA,PASAJEROS,MOTO,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28298,LC-2021-0305042,2021-06-28,2021,6,28,19,14,-58.437400875260,-34.581709500021,CALLE,MOVIL,AUTO,0
28299,LC-2021-0305042,2021-06-28,2021,6,28,19,14,-58.437400875260,-34.581709500021,CALLE,MOVIL,AUTO,0
28301,LC-2021-0451911,2021-09-11,2021,9,11,18,14,-58.420119387377,-34.581370448309,AVENIDA,TRANSPORTE PUBLICO,TRANSPORTE PUBLICO,0
28302,LC-2021-0530228,2021-10-25,2021,10,25,12,14,-58.406897,-34.581142,SD,TRANSPORTE PUBLICO,TRANSPORTE PUBLICO,0


Se pasa FECHA a tipo date

In [9]:
siniestros['FECHA'] = pd.to_datetime(siniestros['FECHA'])

Se pasan año, mes y día a formato datetime

In [10]:
siniestros['A'] = siniestros['FECHA'].dt.year
siniestros['M'] = siniestros['FECHA'].dt.month
siniestros['D'] = siniestros['FECHA'].dt.day

Se crea una nueva columna para el día de la semana

In [11]:
dias_semana_espanol = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
dias_semana_ingles = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
siniestros['DIA_SEMANA'] = siniestros['FECHA'].dt.strftime('%A').map(
    {dia_ingles: dia_espanol for dia_ingles, dia_espanol in zip(dias_semana_ingles, dias_semana_espanol)}
)

***
***

Se pasa LONGITUD y LATITUD a tipo float. (Esto será necesario en el caso de trabajar con folium en Streamlit, no así en Power Bi que reconoce correctamente los datos longitud y latitud cuando se encuentran en tipo texto)

In [12]:
siniestros['LATITUD'] = pd.to_numeric(siniestros['LATITUD'], errors='coerce')
siniestros = siniestros.dropna(subset=['LATITUD'])

In [13]:
siniestros['LONGITUD'] = pd.to_numeric(siniestros['LONGITUD'], errors='coerce')
siniestros = siniestros.dropna(subset=['LONGITUD'])

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3180\2547787231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siniestros['LONGITUD'] = pd.to_numeric(siniestros['LONGITUD'], errors='coerce')


***
***

# Reducción de categorías insignificantes

### Asignación 'OTRO' a los valores de poca aparición en columna ACUSADO

In [14]:
lista_otro = ['MOVIL','CICLISTA','UTILITARIO','MULTIPLE','PEATON','BICICLETA','MONOPATIN','TREN','OTRO']
siniestros = siniestros.copy()
siniestros['ACUSADO'][siniestros['ACUSADO'].isin(lista_otro)] = 'OTRO'
siniestros['ACUSADO'][siniestros['ACUSADO'] == 'TRANSPORTE PUBLICO'] = 'COLECTIVO'

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3180\3812245206.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siniestros['ACUSADO'][siniestros['ACUSADO'].isin(lista_otro)] = 'OTRO'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_3180\3812245206.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siniestros['ACUSADO'][siniestros['ACUSADO'] == 'TRANSPORTE PUBLICO'] = 'COLECTIVO'


Agrupación en ítem CARGAS, a CAMION y CAMIONETA. Agrupación en ítem AUTO a TAXI

In [16]:
siniestros['ACUSADO'][siniestros['ACUSADO'] == 'CAMIONETA'] = 'CARGAS'
siniestros['ACUSADO'][siniestros['ACUSADO'] == 'CAMION'] = 'CARGAS'
siniestros['ACUSADO'][siniestros['ACUSADO'] == 'TAXI'] = 'AUTO'

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3180\122319526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siniestros['ACUSADO'][siniestros['ACUSADO'] == 'CAMIONETA'] = 'CARGAS'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_3180\122319526.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siniestros['ACUSADO'][siniestros['ACUSADO'] == 'CAMION'] = 'CARGAS'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_3180\122319526.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [17]:
siniestros.ACUSADO.unique()

array(['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO', 'OTRO',
       'COLECTIVO'], dtype=object)

### Asignación 'OTRO' a los valores de poca aparición en columna VICTIMA

In [18]:
sin_dato = ['SD','sd']
autos = ['AUTO','TAXI']
bicicleta = ['CICLISTA','BICICLETA','MONOPATIN']
lista_otro = ['MOVIL','MIXTO','CAMION','CARGAS','OTRO','PASAJEROS','UTILITARIO']
siniestros = siniestros.copy()
siniestros['VICTIMA'][siniestros['VICTIMA'].isin(sin_dato)] = 'SD'
siniestros['VICTIMA'][siniestros['VICTIMA'].isin(autos)] = 'AUTO'
siniestros['VICTIMA'][siniestros['VICTIMA'].isin(bicicleta)] = 'BICICLETA' 
siniestros['VICTIMA'][siniestros['VICTIMA'].isin(lista_otro)] = 'OTRO'
siniestros['VICTIMA'][siniestros['VICTIMA'] == 'TRANSPORTE PUBLICO'] = 'COLECTIVO'

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3180\761697361.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siniestros['VICTIMA'][siniestros['VICTIMA'].isin(sin_dato)] = 'SD'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_3180\761697361.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siniestros['VICTIMA'][siniestros['VICTIMA'].isin(autos)] = 'AUTO'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_3180\761697361.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [19]:
siniestros.VICTIMA.value_counts()

VICTIMA
SD           15199
MOTO          4770
PEATON        1854
AUTO          1825
BICICLETA     1741
COLECTIVO      662
OTRO           509
Name: count, dtype: int64

In [20]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26560 entries, 0 to 28303
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          26560 non-null  object        
 1   FECHA       26560 non-null  datetime64[ns]
 2   A           26560 non-null  int32         
 3   M           26560 non-null  int32         
 4   D           26560 non-null  int32         
 5   H           26560 non-null  int64         
 6   COMUNA      26560 non-null  int32         
 7   LONGITUD    26560 non-null  float64       
 8   LATITUD     26560 non-null  float64       
 9   TIPO_CALLE  26560 non-null  object        
 10  ACUSADO     26560 non-null  object        
 11  VICTIMA     26560 non-null  object        
 12  FATAL       26560 non-null  int64         
 13  DIA_SEMANA  26560 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(4), int64(2), object(5)
memory usage: 2.6+ MB


***

### Registros duplicados son posibles y esperables, porque hay hechos con más de una víctima, y no queremos distinguir entre características personales de cada víctima, sino a las estrictamente relacionadas con el hecho. Por esa razón se conservan los registros duplicados, los cuales aportan valor al análisis (suman al contador de víctimas)

In [21]:
siniestros.shape

(26560, 14)

In [22]:
siniestros.drop_duplicates()

,ID,FECHA,A,M,D,H,COMUNA,LONGITUD,LATITUD,TIPO_CALLE,ACUSADO,VICTIMA,FATAL,DIA_SEMANA
0,2016-0001,2016-01-01,2016,1,1,4,8,-58.475340,-34.687570,AVENIDA,AUTO,MOTO,1,Viernes
1,2016-0002,2016-01-02,2016,1,2,1,9,-58.508775,-34.669777,GRAL PAZ,PASAJEROS,AUTO,1,Sábado
2,2016-0003,2016-01-03,2016,1,3,7,1,-58.390403,-34.631894,AVENIDA,AUTO,MOTO,1,Domingo
3,2016-0004,2016-01-10,2016,1,10,0,8,-58.465039,-34.680930,AVENIDA,SD,MOTO,1,Domingo
4,2016-0005,2016-01-21,2016,1,21,5,1,-58.387183,-34.622466,AVENIDA,PASAJEROS,MOTO,1,Jueves
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28293,LC-2021-0314490,2021-07-04,2021,7,4,18,11,-58.516666,-34.589996,CALLE,SD,AUTO,0,Domingo
28296,LC-2021-0230174,2021-05-26,2021,5,26,11,14,-58.431960,-34.580349,CALLE,SD,SD,0,Miércoles
28297,LC-2021-0305042,2021-06-28,2021,6,28,19,14,-58.437401,-34.581710,CALLE,OTRO,AUTO,0,Lunes
28301,LC-2021-0451911,2021-09-11,2021,9,11,18,14,-58.420119,-34.581370,AVENIDA,COLECTIVO,COLECTIVO,0,Sábado


***

### Las visualizaciones para verificación de outliers se dan naturalmente en el archivo visualizaciones (luego de los KPI), ya que todas las variables son gráficadas con entendimiento del problema, y se detectarían de manera clara cualquier tipo de fecha o rango horario por fuera de lo esperado.

### Las columnas con las que trabajamos ya fueron revisadas en cuanto a los valores "atípicos", y fueron corregidos los errores encontrados

***
***

### Exportación de archivo siniestrosEDA.csv

In [23]:
siniestros.to_csv('siniestrosEDA.csv', index=False)